In [1]:
#refer https://www.kaggle.com/waxbabi/light-gbm-benchmark-0-3692/code
#implement basic content-based prediction

In [2]:
import pandas as pd
import numpy as np
from scipy import sparse

In [3]:
products = pd.read_csv('../csv/products.csv')

In [4]:
order_products_prior = pd.read_csv('../csv/order_products__prior.csv')

In [5]:
order_products_train = pd.read_csv('../csv/order_products__train.csv')

In [6]:
orders = pd.read_csv('../csv/orders.csv')

In [7]:
df = order_products_prior.set_index('order_id').join(orders.set_index('order_id'))

In [8]:
product_user = pd.DataFrame(df.groupby(['product_id', 'user_id']).size().rename('count')).reset_index()

In [9]:
truncted_product_user = product_user.copy()
truncted_product_user.loc[truncted_product_user['count'] > 10, 'count'] = 10

In [10]:
product_level = pd.pivot_table(truncted_product_user, index=['product_id'], columns=['count'], aggfunc=len, fill_value=0)\
.reset_index().set_index('product_id').add_prefix('plevel_')

In [11]:
user_level = pd.pivot_table(truncted_product_user, index=['user_id'], columns=['count'], aggfunc=len, fill_value=0)\
.reset_index().set_index('user_id').add_prefix('ulevel_')

In [12]:
product_v = df.groupby('product_id').agg({'user_id': ['nunique'], 'reordered': ['sum', 'count']})\
.reset_index().set_index('product_id').add_prefix('p_')

In [13]:
user_v = df.groupby('user_id').agg({'product_id': ['nunique'], 'reordered': ['sum', 'count']})\
.reset_index().set_index('user_id').add_prefix('u_')

In [14]:
label = orders.set_index('order_id').join(order_products_train.set_index('order_id'), how='inner')[['user_id', 'product_id']]
label['label'] = 1

In [16]:
products[['aisle_id', 'department_id']].head()

,aisle_id,department_id
0,61,19
1,104,13
2,94,7
3,38,1
4,5,13


In [17]:
train = orders[orders.eval_set == 'train'].set_index('user_id').join(product_user.set_index('user_id'))\
.reset_index().drop('order_id', 1).set_index(['user_id', 'product_id'])\
.join(label.reset_index().drop('order_id', 1).set_index(['user_id', 'product_id']))\
.reset_index().set_index('product_id').join(product_level).join(product_v)\
.join(products[['product_id', 'aisle_id', 'department_id']].set_index('product_id'))\
.reset_index().set_index('user_id').join(user_level).join(user_v)\
.reset_index().fillna(0)

/home/zyc/anaconda2/lib/python2.7/site-packages/pandas/tools/merge.py:205: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [18]:
train.to_csv('../data/train.data')

In [19]:
test = orders[orders.eval_set == 'test'].set_index('user_id').join(product_user.set_index('user_id'))\
.drop('order_id', 1)\
.reset_index().set_index('product_id').join(product_level).join(product_v)\
.join(products[['product_id', 'aisle_id', 'department_id']].set_index('product_id'))\
.reset_index().set_index('user_id').join(user_level).join(user_v)\
.reset_index().fillna(0)

In [20]:
test.to_csv('../data/test.data')